In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

In [4]:
path = '/Users/thudson/Documents/Dropbox/chicago-police-data/import/output'
folders = [folder for folder in os.listdir(path) if '.' not in folder]
folders

['complaints-cpd-2016-dec_copy_20170112',
 'complaints-cpd-2016-jun_copy_20170112',
 'complaints-cpd-2016-nov_copy_20170112',
 'complaints-cpd-2016-oct_copy_20170112',
 'complaints-ipra-2016-apr_copy_20170112',
 'complaints-merged-2015_copy_20170112',
 'shootings-cpd-feb2016_copy_20170112',
 'shootings-ipra-may2016_copy_20170112',
 'TRRs']

In [9]:
folder = folders[0]
files = [file for file in os.listdir('/'.join([path,folder])) 
         if '.csv' in file and 'metadata' not in file]
files

['MAINFRAME_COMPLAINTS_REVISED_30NOV2016.csv',
 'NEW_WITNESS_FILE_NOV_29_2016_-_no_emp_number.csv']

In [11]:
file = files[0]
filepath = '/'.join([path,folder,file])
df = pd.read_csv(filepath)
df.head()

/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,8,11,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CRID,Incident_Date,Incident_Time,Complaint_Date,Case_Status,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Beat,Address_of_Incident,...,Accused_Birth_Year,Accused_Appointed_Date,Current_Unit,Final_Complaint_Category,Current_Rank,Complaint_Description,Final_Finding,Final_Finding_Description,Final_Action_Taken,Final_Action_Description
0,C035735,10/21/1967,1400,10/21/1967,CLO,8/1/1980,RICHARD ROCHFORD,142,6330,90** S. COTTAGE GROVE,...,NaN,9/21/1953,171,10Y,9161,OPERATION/PERSONNEL VIOL.: ACT TO CIRCUMVENT ...,SU,SUSTAINED,30,30 DAY SUSPENSION
1,C050876,7/4/1970,2315,8/13/1970,CLO,9/15/1973,EDWARD DOWNES,GIS,1833,"HOLIDAY INN,6** N LAKE SHORE DRIVE",...,NaN,7/22/1968,17,06E,9161,BRIBERY/OFFICIAL CORRUPTION: GRATUITY,SU,SUSTAINED,200,SUSPENDED OVR 30 DAY
2,C051271,2/18/1972,1600,2/18/1972,CLO,2/2/1973,LEONARD ZALESKI,SIS,3100,80** S NEWLAND OAK LAWN IL,...,NaN,5/27/1968,192,10R,9161,OPERATION/PERSONNEL VIOLATION: RESIDENCY,SU,SUSTAINED,200,SUSPENDED OVR 30 DAY
3,C051597,9/22/1970,315,9/22/1970,CLO,10/15/1971,MICHAEL HALKO,8,824,MARQUETTE PARK,...,NaN,2/27/1967,22,02B,9161,ALCOHOL ABUSE: INTOXICATED OFF DUTY,SU,SUSTAINED,200,SUSPENDED OVR 30 DAY
4,C052045,10/14/1970,30,10/19/1970,CLO,3/9/1987,RICHARD VOSS,GIS,1822,18TH DIST,...,NaN,11/14/1966,641,06E,9165,BRIBERY/OFFICIAL CORRUPTION: GRATUITY,SU,SUSTAINED,200,SUSPENDED OVR 30 DAY


In [16]:
df.columns

Index(['CRID', 'Incident_Date', 'Incident_Time', 'Complaint_Date',
       'Case_Status', 'Closed_Date', 'Investigator_Full_Name',
       'Investigator_Assignment', 'Beat', 'Address_of_Incident', 'Accused',
       'Star', 'Accused_Gender', 'Accused_Birth_Year',
       'Accused_Appointed_Date', 'Current_Unit', 'Final_Complaint_Category',
       'Current_Rank', 'Complaint_Description', 'Final_Finding',
       'Final_Finding_Description', 'Final_Action_Taken',
       'Final_Action_Description'],
      dtype='object')

### Seems like there are repeated rows, need to figure out what information, if any, is unique

In [50]:
df.groupby(['CRID','Accused','Beat']).Beat.count().head(20)

CRID     Accused           Beat
C035735  FRANK LYNCH       6330    1
C050876  PETER PARISI      1833    1
C051271  JAMES AHERN       3100    1
C051597  JOHN HAUGH        824     1
C052045  ROBERT BROWNE     1822    1
C052094  STANLEY JASPER    1224    1
C053661  CRAIG CEGIELSKI   905     2
         DENNIS MULCAHY    905     1
C054286  SAMUEL PESOLI     113     1
         TONY MUGNOLO      113     1
C056104  DON ROBERTS       384     1
C058189  THOMAS CHIGAROS   806     1
C058264  WILLIAM SMITH     1134    1
C093392  DENNIS MC NAMARA  2306    1
C100256  ROBERT SHANAHAN   1442    1
C102595  JAMES SENG        1623    1
C104195  JAMES SMITH JR    1116    1
C106081  JAMES SMITH JR    221     1
C106227  JOHN SASSO        1642    1
C106520  JAMES SMITH JR    221     1
Name: Beat, dtype: int64

In [52]:
df[df['Accused']=='CRAIG CEGIELSKI'].head()

,CRID,Incident_Date,Incident_Time,Complaint_Date,Case_Status,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Beat,Address_of_Incident,...,Accused_Birth_Year,Accused_Appointed_Date,Current_Unit,Final_Complaint_Category,Current_Rank,Complaint_Description,Final_Finding,Final_Finding_Description,Final_Action_Taken,Final_Action_Description
7,C053661,2/17/1971,19,2/17/1971,CLO,7/17/1987,EUGENE WISNIEWSKI,GIS,905,39** S. KEDZIE AVE,...,NaN,5/24/1999,632,05G,9165,"EXCESSIVE FORCE: WEAPON, USE OF",SU,SUSTAINED,200,SUSPENDED OVR 30 DAY
8,C053661,2/17/1971,19,2/17/1971,CLO,7/17/1987,EUGENE WISNIEWSKI,GIS,905,39** S. KEDZIE AVE,...,NaN,5/12/1969,632,05G,9165,"EXCESSIVE FORCE: WEAPON, USE OF",SU,SUSTAINED,200,SUSPENDED OVR 30 DAY
1430,C147895,10/17/1985,900,10/17/1985,CLO,1/26/1987,JAMES MONTGOMERY,OPS,231,AREA 1 DETECTIVE DIVISION,...,NaN,5/12/1969,632,05B,9165,EXCESSIVE FORCE: ARRESTEE - AFTER ARREST-PRIO...,NS,NOT SUSTAINED,600,NO ACTION TAKEN
1431,C147895,10/17/1985,900,10/17/1985,CLO,1/26/1987,JAMES MONTGOMERY,OPS,231,AREA 1 DETECTIVE DIVISION,...,NaN,5/24/1999,632,05B,9165,EXCESSIVE FORCE: ARRESTEE - AFTER ARREST-PRIO...,NS,NOT SUSTAINED,600,NO ACTION TAKEN
9354,C172652,12/6/1989,530,12/7/1989,CLO,2/13/1990,FRANCIS LEE JR,8,822,35** W 63RD STREET,...,NaN,5/12/1969,632,10J,9165,OPERATION/PERSONNEL VIOLS.: NEGLECT OF DUTY /...,NS,NOT SUSTAINED,600,NO ACTION TAKEN


In [14]:
len(df.CRID.unique())

70302

### Comparing Dec to Nov

In [34]:
folder = folders[2]
files = [file for file in os.listdir('/'.join([path,folder])) 
         if '.csv' in file and 'metadata' not in file]
files

['p046957_-_report_1_-_all_complaints_in_time_frame.csv',
 'p046957_-_report_2_-_identified_accused.csv',
 'p046957_-_report_3_-_police_officer_witness_data_xi.csv',
 'p046957_-_report_5_-_complainant_(reporting_party)_data.csv']

In [35]:
file = files[0]
filepath = '/'.join([path,folder,file])
df1 = pd.read_csv(filepath)
df1.head()

/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CRID,Beat,Location_Code,Location_Value,Address_of_Incident,City,State,Zip,Incident_Date,Complaint_Date,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Investigator_Rank,Investigator_Star,Investigator_Appt_Date,FOIA_Request_Number
0,258996,1524,04,Police Building,5327 W CHICAGO,CHICAGO,IL,NaN,2000-01-01 01:20:00,2000-01-01 00:00:00,2001-01-26 00:00:00,"SCHWIEGER, STEVEN",13.0,LIEUTENANT OF POLICE,NaN,1986-06-16 00:00:00,FOIA P046957\nReport 1.1\nAll Complaints in CR...
1,258997,1115,17,Public Way - Other,4316 W JACKSON,CHICAGO,IL,NaN,2000-01-01 01:30:00,2000-01-01 00:00:00,2000-10-14 00:00:00,"MULLIGAN JR, MICHAEL",8.0,SERGEANT OF POLICE,NaN,1970-12-14 00:00:00,FOIA P046957\nReport 1.1\nAll Complaints in CR...
2,258998,1834,17,Public Way - Other,500 W ILLINOIS,CHICAGO,IL,NaN,2000-01-01 00:28:00,2000-01-01 00:00:00,2001-01-18 00:00:00,"MC MAHON, MAUREEN",608.0,LIEUTENANT OF POLICE,NaN,1985-07-01 00:00:00,FOIA P046957\nReport 1.1\nAll Complaints in CR...
3,258999,0,17,Public Way - Other,NaN,CHICAGO,IL,NaN,2000-01-01 03:30:00,2000-01-01 00:00:00,2000-03-23 00:00:00,"DEAN, BRUCE",113.0,SUPERVISING INV IPRA,NaN,1995-09-16 00:00:00,FOIA P046957\nReport 1.1\nAll Complaints in CR...
4,259000,1524,04,Police Building,5327 W CHICAGO AVE,CHICAGO,IL,NaN,2000-01-01 05:00:00,2000-01-01 00:00:00,2001-01-17 00:00:00,"LABERN, LINDA",20.0,SERGEANT OF POLICE,NaN,1973-07-16 00:00:00,FOIA P046957\nReport 1.1\nAll Complaints in CR...


In [36]:
df1.columns

Index(['CRID', 'Beat', 'Location_Code', 'Location_Value',
       'Address_of_Incident', 'City', 'State', 'Zip', 'Incident_Date',
       'Complaint_Date', 'Closed_Date', 'Investigator_Full_Name',
       'Investigator_Assignment', 'Investigator_Rank', 'Investigator_Star',
       'Investigator_Appt_Date', 'FOIA_Request_Number'],
      dtype='object')

In [37]:
df1.shape

(138350, 17)

In [38]:
len(df1.CRID.unique())

109339

In [42]:
len(df1.groupby(['CRID','Investigator_Full_Name','Incident_Date']).Beat.count())

108742

In [44]:
df1 = df1.sort_values(by=['CRID','Investigator_Full_Name','Investigator_Appt_Date'])

In [45]:
df1.head()

,CRID,Beat,Location_Code,Location_Value,Address_of_Incident,City,State,Zip,Incident_Date,Complaint_Date,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Investigator_Rank,Investigator_Star,Investigator_Appt_Date,FOIA_Request_Number
119135,106213,1631,17,Public Way - Other,3700 N HARLEM AVE,CHICAGO,IL,60634,2015-07-19 21:00:00,2015-07-20 00:00:00,2015-09-21 00:00:00,"KLIMAS, ROBERT",121.0,COMMANDER,0.0,2008-08-04 00:00:00,FOIA P046957\nReport 1.6\nAll Complaints in CR...
119136,107899,611,17,Public Way - Other,7843 S HERMITAGE AVE,CHICAGO,IL,6062,2015-08-16 20:20:00,2016-01-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1.6\nAll Complaints in CR...
119137,107901,1814,17,Public Way - Other,1622 N NORTH PARK AVE,CHICAGO,IL,60614,2016-01-25 21:50:00,2016-01-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1.6\nAll Complaints in CR...
119138,108026,1722,04,Police Building,4650 N PULASKI RD,CHICAGO,IL,NaN,2013-10-14 00:01:00,2016-04-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1.6\nAll Complaints in CR...
119139,108059,0,XX,NaN,NaN,CHICAGO,IL,NaN,2016-05-12 00:00:00,2016-05-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1.6\nAll Complaints in CR...


In [47]:
## NO CRID IN COMMON BETWEEN DEC FILE AND NOV COMPLAINT FILE
df.merge(df1,how='inner',on='CRID')

,CRID,Incident_Date_x,Incident_Time,Complaint_Date_x,Case_Status,Closed_Date_x,Investigator_Full_Name_x,Investigator_Assignment_x,Beat_x,Address_of_Incident_x,...,Zip,Incident_Date_y,Complaint_Date_y,Closed_Date_y,Investigator_Full_Name_y,Investigator_Assignment_y,Investigator_Rank,Investigator_Star,Investigator_Appt_Date,FOIA_Request_Number


### Looking at Officer Files

In [54]:
file = files[1]
filepath = '/'.join([path,folder,file])
df1_1 = pd.read_csv(filepath)
df1_1.head()

/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CRID,Accused,Accused_Birth_Year,Accused_Gender,Accused_Race_Code,Date_of_Appt,Current_Unit,Current_Rank,Star,Complaint_Category,Orig_Finding,Orig_Recommended_Discipline,Final_Finding,Final_Recommended_Discipline,FOIA_Request_Number,Report_Produced_Date
0,258996,"BARRON, WILLIAM",1949.0,M,WHI,1978-02-27,018,SGT,NaN,01A-USE OF PROFANITY,NS,600.0,NS,600.0,FOIA P046957\nReport 2.1\nAll Complaints in CR...,NaN
1,258997,"C0NNOLLY, KIMBERLY",1965.0,F,BLK,1990-07-30,055,NaN,11026.0,01A-USE OF PROFANITY,UN,600.0,UN,600.0,FOIA P046957\nReport 2.1\nAll Complaints in CR...,NaN
2,258997,"KEENE, JOHN",1968.0,M,WHI,1999-03-08,153,PO,NaN,01A-USE OF PROFANITY,UN,600.0,UN,600.0,FOIA P046957\nReport 2.1\nAll Complaints in CR...,NaN
3,258998,"SLAVIN, SCOTT",1965.0,M,WHI,1991-11-18,145,SGT,807.0,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,EX,600.0,EX,600.0,FOIA P046957\nReport 2.1\nAll Complaints in CR...,NaN
4,259001,"MARTINEZ, ANTONIO",1971.0,M,S,1996-11-04,701,PO,NaN,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,UN,600.0,UN,600.0,FOIA P046957\nReport 2.1\nAll Complaints in CR...,NaN


In [55]:
## NO CRID IN COMMON BETWEEN DEC FILE AND NOV OFFICER FILE
df.merge(df1_1,how='inner',on='CRID')

,CRID,Incident_Date,Incident_Time,Complaint_Date,Case_Status,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Beat,Address_of_Incident,...,Current_Unit_y,Current_Rank_y,Star_y,Complaint_Category,Orig_Finding,Orig_Recommended_Discipline,Final_Finding_y,Final_Recommended_Discipline,FOIA_Request_Number,Report_Produced_Date


In [113]:
## ACCUSED NAMES IN COMMON BETWEEN DEC FILE AND NOV FILE
names_nov = [x[1]+' '+x[0] for x in df1_1['Accused'].str.split(', ')]
names_dec = df['Accused'].tolist()

In [98]:
names_nov[0]

'WILLIAM BARRON'

In [97]:
matching_names = [x for x in names_nov if x in names_dec]
matching_names[0:10]

['WILLIAM BARRON',
 'JOHN KEENE',
 'SCOTT SLAVIN',
 'ANTONIO MARTINEZ',
 'MICHAEL JONES',
 'CORNELIUS BROWN',
 'KENNETH MOLESKY',
 'SOFIA TERRONES',
 'JAMES BRILL',
 'JOSEPH CRAIG']

In [99]:
len(matching_names)

78952

In [56]:
## NO CRID IN COMMON BETWEEN DEC FILE AND NOV OFFICER FILE
df.merge(df1_1,how='inner',on='Accused')

,CRID_x,Incident_Date,Incident_Time,Complaint_Date,Case_Status,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Beat,Address_of_Incident,...,Current_Unit_y,Current_Rank_y,Star_y,Complaint_Category,Orig_Finding,Orig_Recommended_Discipline,Final_Finding_y,Final_Recommended_Discipline,FOIA_Request_Number,Report_Produced_Date


### October, Nov, Dec

In [100]:
folder = folders[3]
files = [file for file in os.listdir('/'.join([path,folder])) 
         if '.csv' in file and 'metadata' not in file]
files

['CR_AllRecords.csv']

In [101]:
file = files[0]
filepath = '/'.join([path,folder,file])
df2 = pd.read_csv(filepath)
df2.head()

/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Officer_Name,Officer_Unit,Date_of_Appointment,CRID,Initial_Complaint_Category,Final_Complaint_Category,Incident_Date,Complaint_Date,Closed_Date,Final_Finding,Final_Action_Taken,Final_Action_Description
0,"ABBATE, ANTHONY G",011,05-DEC-1994,237004,05A ARRESTEE - DURING ARREST,05B ARRESTEE - DURING ARREST,21-MAY-1997,21-MAY-1997,25-MAY-1999,UN,600,NO ACTION TAKEN
1,"ABBATE, ANTHONY G",011,05-DEC-1994,258124,04E PRISONERS PROPERTY - INVENTORY/RECEIPT,04E PRISONERS PROPERTY - INVENTORY/RECEIPT,19-NOV-1999,20-NOV-1999,07-DEC-2000,NS,600,NO ACTION TAKEN
2,"ABBATE, CARMEL G",641,06-JAN-1969,190369,03G MISCELLANEOUS,03G MISCELLANEOUS,11-FEB-1992,11-FEB-1992,09-APR-1992,UN,600,NO ACTION TAKEN
3,"ABBATE, CARMEL G",640,06-JAN-1969,210426,10V INVENTORY PROCEDURES,04E INVENTORY PROCEDURES,30-JUL-1994,14-JUL-1994,01-DEC-1994,NS,600,NO ACTION TAKEN
4,"ABBATE, CARMEL G",640,06-JAN-1969,219424,09J MISCELLANEOUS,05A MISCELLANEOUS,12-JUL-1995,19-JUL-1995,23-JAN-1996,NS,600,NO ACTION TAKEN


In [104]:
## NO CRID IN COMMON BETWEEN DEC FILE AND OCT OFFICER FILE
df.merge(df2,how='inner',on='CRID')

,CRID,Incident_Date_x,Incident_Time,Complaint_Date_x,Case_Status,Closed_Date_x,Investigator_Full_Name,Investigator_Assignment,Beat,Address_of_Incident,...,Officer_Unit,Date_of_Appointment,Initial_Complaint_Category,Final_Complaint_Category_y,Incident_Date_y,Complaint_Date_y,Closed_Date_y,Final_Finding_y,Final_Action_Taken_y,Final_Action_Description_y


In [111]:
## ACCUSED NAMES IN COMMON BETWEEN DEC FILE AND OCT FILE
names_oct = [x[1]+' '+x[0] for x in df2['Officer_Name'].str.split(', ')]

In [114]:
matching_names = [x for x in names_oct if x in names_dec]
matching_names[0:10]

['RASHAD ABDULLAH',
 'RASHAD ABDULLAH',
 'ROBERT ABERNATHY',
 'JOHN ABRAMS',
 'JOHN ABRAMS',
 'JOHN ABRAMS',
 'JOHN ABRAMS',
 'JOHN ABRAMS',
 'JOHN ABRAMS',
 'JOHN ABRAMS']

In [106]:
## CRID IN COMMON BETWEEN NOV FILE AND OCT OFFICER FILE
df1.merge(df2,how='inner',on='CRID').shape

(12437, 28)

### June, Oct, Nov, Dec

In [115]:
folder = folders[1]
files = [file for file in os.listdir('/'.join([path,folder])) 
         if '.csv' in file and 'metadata' not in file]
files

['P046957_-_report_1_-_all_complaints_in_time_frame-unlocked.csv',
 'P046957_-_report_2_-_identified_accused_xi-unlocked.csv',
 'P046957_-_report_3_-_police_officer_witness_data_xi-unlocked.csv',
 'P046957_-_report_4_-_victim_data-unlocked.csv',
 'P046957_-_report_5_-_complainant_(reporting_party)_data-unlocked.csv']

In [116]:
file = files[0]
filepath = '/'.join([path,folder,file])
df3 = pd.read_csv(filepath)
df3.head()

,CRID,Beat,Location_Code,Location_Value,Address_of_Incident,City,State,Zip,Incident_Date,Complaint_Date,Closed_Date,Investigator_Full_Name,Investigator_Assignment,Investigator_Rank,Investigator_Star,Investigator_Appt_Date,FOIA_Request_Number,Report_Produced_Date
0,106213,1631.0,17,Public Way - Other,3700 N HARLEM AVE,CHICAGO,IL,60634,2015-07-19 21:00:00,2015-07-20 00:00:00,2015-09-21 00:00:00,"KLIMAS, ROBERT",121.0,COMMANDER,0.0,2008-08-04 00:00:00,FOIA P046957\nReport 1\nAll Complaints in CRMS...,NaN
1,107899,611.0,17,Public Way - Other,7843 S HERMITAGE AVE,CHICAGO,IL,6062,2015-08-16 20:20:00,2016-01-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1\nAll Complaints in CRMS...,NaN
2,107901,1814.0,17,Public Way - Other,1622 N NORTH PARK AVE,CHICAGO,IL,60614,2016-01-25 21:50:00,2016-01-26 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1\nAll Complaints in CRMS...,NaN
3,108026,1722.0,04,Police Building,4650 N PULASKI RD,CHICAGO,IL,NaN,2013-10-14 00:01:00,2016-04-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1\nAll Complaints in CRMS...,NaN
4,108109,832.0,17,Public Way - Other,2340 W 69TH ST,CHICAGO,IL,NaN,2016-02-26 11:00:00,2016-04-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,FOIA P046957\nReport 1\nAll Complaints in CRMS...,NaN


In [121]:
## NO CRID IN COMMON BETWEEN DEC FILE AND JUNE FILE
df.merge(df3,how='inner',on='CRID').shape

(0, 40)

In [119]:
## CRID IN COMMON BETWEEN NOV FILE AND JUNE FILE
df1.merge(df3,how='inner',on='CRID').shape

(18111, 34)

In [120]:
## NO CRID IN COMMON BETWEEN OCT FILE AND JUNE FILE
df2.merge(df3,how='inner',on='CRID').shape

(0, 29)